In [172]:
# Run in terminal or command prompt:
#python3 -m spacy download en

# Packages
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pprint import pprint

# Import stopwords and other word packages
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LdaMulticore
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models # as gensimvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import stopwords

from scipy.sparse import csr_matrix, hstack, coo_matrix


import string
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mattparker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mattparker/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mattparker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [173]:
airbnb = pd.read_csv('../data/airbnb_gentrification.csv')

In [174]:
airbnb.head(1)

,listing_id,comments_concatenated,name,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,listing_url,description,neighborhood_overview,host_since,host_listings_count,property_type,accommodates,bathrooms_text,bedrooms,beds,amenities,minimum_nights_avg_ntm,maximum_nights_avg_ntm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,zip_code,GEOID,house_price_2021-01-31,house_pct_change,rentals_2021-01-31,rental_price_pct_change,new_restaurants,available_beer,str_permits_2020,str_permits_growth,crimes,total_pop_2010,total_pop_2019,total_pop_change,total_pop_pct_change,pop_over25_2010,pop_over25_2019,pop_over25_change,pop_over25_pcg_change,total_households_2010,total_households_2019,total_households_change,total_households_pct_change,white_pct_2010,white_pct_2019,white_value_change,white_pct_change,bach_pct_2010,bach_pct_2019,bach_value_change,bach_pct_change,rent_pct_2010,rent_pct_2019,rent_value_change,renter_pct_change,median_hhi_2010,median_hhi_2019,median_hhi_value_change,median_hhi_pct_change,poverty_pct_2010,poverty_pct_2019,poverty_value_change,poverty_pct_change,gentrifying
0,6422,I can't say enough about how wonderful it was ...,Nashville Charm,12172,36.17315,-86.73581,40,30,674,4.69,1,267,https://www.airbnb.com/rooms/6422,30 day or more rental during COVID. Show COVID...,Historic East Nashville is home to many new an...,2009-04-03,0.0,Private room in house,2,1 private bath,2.0,3.0,"[""Hair dryer"", ""Bathtub"", ""Lock on bedroom doo...",30.0,365.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,37206.0,4.703701e+10,412476.0,38.31,NaN,NaN,1.0,2.0,114.0,114.0,1165.0,2544.0,2100.0,-444.0,-0.174528,1703.0,1639.0,-64.0,-0.037581,1140.0,926.0,-214.0,-0.187719,0.657626,0.940952,0.283327,0.430833,0.408691,0.585723,0.177032,0.43317,0.320175,0.240821,-0.079355,-0.247848,46000.0,91643.0,45643.0,0.992239,10.6,10.2,-0.4,-0.037736,False


In [175]:
airbnb.shape

(5205, 76)

In [176]:
airbnb = airbnb[airbnb['comments_concatenated'].notna()]

In [177]:
airbnb_nlp = airbnb[['listing_id', 'comments_concatenated', 'gentrifying']]

In [178]:
airbnb_nlp.comments_concatenated.isna().sum()

0

## Train Test Split (skip unless using a predictive model)

In [179]:
# create two lists with training and testing apns
train_listings, test_listings = tts(airbnb_nlp['listing_id'].to_list(), 
                                    random_state = 42, 
                                    stratify=airbnb_nlp['gentrifying'])

In [180]:
len(train_listings)

3903

In [181]:
# Create train and test dataframes from the lists of apns
airbnb_train = airbnb_nlp[airbnb_nlp['listing_id'].isin(train_listings)].sort_values('listing_id')
airbnb_test = airbnb_nlp[airbnb_nlp['listing_id'].isin(test_listings)].sort_values('listing_id')

In [182]:
# Create the y_train and y_test dataframes from the lists of apns
y_train = airbnb_nlp['gentrifying']
y_test = airbnb_nlp['gentrifying']

In [183]:
print(airbnb_train.gentrifying.value_counts(normalize=True))
print(airbnb_test.gentrifying.value_counts(normalize=True))

False    0.755829
True     0.244171
Name: gentrifying, dtype: float64
False    0.756341
True     0.243659
Name: gentrifying, dtype: float64


## Gensim Blog - Machine Learning Plus
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [260]:
# Convert to list
data = airbnb_nlp.comments_concatenated.values.tolist()

In [261]:
# Clean review break symbols
data = [re.sub("\\r\\n", "", comment) for comment in data]

In [262]:
# Tokenize each sentence to words, removing uneeded words/characters
def sent_to_words(sentences):
    for sentence in tqdm(sentences):
        tagged_words = nltk.tag.pos_tag(sentence.split()) 
        no_names = [word for word,tag in tagged_words if tag != 'NNP' and tag != 'NNPS'] # Remove proper nouns
        yield(gensim.utils.simple_preprocess(str(no_names), deacc=True)) #Clean and remove punctuation

data_words = list(sent_to_words(data))

#print(data_words[0:1])

In [263]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words,
                               min_count=100,
                               threshold=10)#,  # higher threshold fewer phrases.
                               #connector_words=phrases.ENGLISH_CONNECTOR_WORDS) ***I think I need to download this.
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)#, connector_words=phrases.ENGLISH_CONNECTOR_WORDS)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Test trigram on first review
#print(trigram_mod[bigram_mod[data_words[0]]])

In [264]:
# Build list of stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'stay', 'place', 'location', 'home', 'house', 'host', 'hosts', 'great', 'clean', 'nice',
                  'perfect', 'comfortable', 'room', 'definitely', 'recommend', 'space', 'need', 'us', 'would',
                  'building', 'good', 'get', 'one', 'everything', 'loved'])

In [265]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

In [266]:
# Only use if skipping lemmatization
data_lemmatized = data_words_bigrams.copy()

In [267]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tqdm(texts)]

# View
#print(corpus[:1])

In [268]:
# How to view a single word within the corpus
id2word[500]

'conversation'

In [269]:
# Human readable format of corpus (term-frequency)
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [270]:
# Build LDA model
lda_model = LdaMulticore(corpus=corpus,        # Replace with gensim.models.ldamodel.LdaModel()
                       id2word=id2word,
                       num_topics=8, #number of topics to identify
                       random_state=100,
                       #update_every=1,                          #Add back in with LdaModel
                       chunksize=100, #number of documents to pass per chunk
                       passes=10, #number of training passes
                       #alpha='auto',                            #Add back in with LdaModel
                       per_word_topics=True)

In [271]:
# Print the top 10 Keywords in each grouped Topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"downtown" + 0.015*"beds" + 0.014*"group" + 0.012*"amazing" + '
  '0.011*"beautiful" + 0.010*"spacious" + 0.010*"time" + 0.010*"close" + '
  '0.010*"quick" + 0.009*"plenty"'),
 (1,
  '0.018*"friendly" + 0.012*"welcoming" + 0.012*"bathroom" + 0.011*"made" + '
  '0.011*"staying" + 0.010*"time" + 0.010*"wonderful" + 0.009*"private" + '
  '0.009*"experience" + 0.009*"beautiful"'),
 (2,
  '0.073*"condo" + 0.057*"view" + 0.034*"downtown" + 0.030*"walk" + '
  '0.022*"river" + 0.018*"views" + 0.015*"balcony" + 0.015*"close" + '
  '0.014*"pool" + 0.010*"city"'),
 (3,
  '0.013*"super" + 0.012*"cute" + 0.012*"amazing" + 0.010*"cozy" + '
  '0.010*"little" + 0.010*"downtown" + 0.009*"stylish" + 0.009*"staying" + '
  '0.009*"close" + 0.008*"bed"'),
 (4,
  '0.022*"restaurants" + 0.014*"neighborhood" + 0.013*"downtown" + '
  '0.012*"walk" + 0.011*"easy" + 0.010*"apartment" + 0.009*"area" + '
  '0.009*"close" + 0.008*"bars" + 0.008*"time"'),
 (5,
  '0.020*"apartment" + 0.009*"close" + 0.0

In [272]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score - Likely more helpful. Takes a while to run.
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.166391972531052

Coherence Score:  0.3107761181720091


## Visual for Viewing each Topic

In [284]:
#vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, sort_topics=False)
#pyLDAvis.save_html(vis, '../lda_vis/lda.html')
# To publish to google site, add the following to front of URL: https://htmlpreview.github.io/?

### Skipping step 17 about finding the best number of topics - Tim recommends 8-12

## Find dominant Topic in each Review

In [274]:
bow = corpora.Dictionary(data_lemmatized)

In [275]:
#Michael's Original
contents = []

for idx, doc in tqdm(enumerate(data_lemmatized)):
    bow = id2word.doc2bow(doc)
    topics = lda_model.get_document_topics(bow)
    for res in topics:
        topic, pct = res
        contents.append({'index': idx, 'topic': topic, 'percent': pct})

topics = pd.DataFrame(contents)

In [276]:
topics.head(10)

,index,topic,percent
0,0,1,0.509356
1,0,4,0.168034
2,0,6,0.178528
3,0,7,0.127615
4,1,1,0.441536
5,1,4,0.406990
6,1,5,0.027662
7,1,6,0.095980
8,1,7,0.023219
9,2,1,0.127599


In [277]:
idx = topics.groupby(['topic'])['percent'].transform(max) == topics['percent']
topic_listing = topics[idx]
topic_listing

,index,topic,percent
2078,413,2,0.937429
5808,1185,0,0.997171
6948,1427,1,0.967497
16767,3685,4,0.986514
19943,4516,7,0.982811
20503,4668,3,0.979470
21228,4868,6,0.982094
22050,5078,5,0.997293


In [278]:
idx = topics.groupby(['index'])['percent'].transform(max) == topics['percent']
hot_topics = topics[idx].set_index('index')
hot_topics.head()

,topic,percent
index,,
0,1,0.509356
1,1,0.441536
2,6,0.495062
3,1,0.460281
4,1,0.417479


In [279]:
hot_topics.value_counts('topic').sort_index()

topic
0    1514
1     142
2     100
3     636
4     608
5    1064
6     346
7     962
dtype: int64

In [280]:
airbnb_topics = hot_topics.join(airbnb)

In [281]:
airbnb_topics.head()

,topic,percent,listing_id,comments_concatenated,name,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,listing_url,description,neighborhood_overview,host_since,host_listings_count,property_type,accommodates,bathrooms_text,bedrooms,beds,amenities,minimum_nights_avg_ntm,maximum_nights_avg_ntm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,zip_code,GEOID,house_price_2021-01-31,house_pct_change,rentals_2021-01-31,rental_price_pct_change,new_restaurants,available_beer,str_permits_2020,str_permits_growth,crimes,total_pop_2010,total_pop_2019,total_pop_change,total_pop_pct_change,pop_over25_2010,pop_over25_2019,pop_over25_change,pop_over25_pcg_change,total_households_2010,total_households_2019,total_households_change,total_households_pct_change,white_pct_2010,white_pct_2019,white_value_change,white_pct_change,bach_pct_2010,bach_pct_2019,bach_value_change,bach_pct_change,rent_pct_2010,rent_pct_2019,rent_value_change,renter_pct_change,median_hhi_2010,median_hhi_2019,median_hhi_value_change,median_hhi_pct_change,poverty_pct_2010,poverty_pct_2019,poverty_value_change,poverty_pct_change,gentrifying
0,1,0.509356,6422.0,I can't say enough about how wonderful it was ...,Nashville Charm,12172.0,36.17315,-86.73581,40.0,30.0,674.0,4.69,1.0,267.0,https://www.airbnb.com/rooms/6422,30 day or more rental during COVID. Show COVID...,Historic East Nashville is home to many new an...,2009-04-03,0.0,Private room in house,2.0,1 private bath,2.0,3.0,"[""Hair dryer"", ""Bathtub"", ""Lock on bedroom doo...",30.0,365.0,99.0,10.0,10.0,10.0,10.0,10.0,10.0,37206.0,4.703701e+10,412476.0,38.31,NaN,NaN,1.0,2.0,114.0,114.0,1165.0,2544.0,2100.0,-444.0,-0.174528,1703.0,1639.0,-64.0,-0.037581,1140.0,926.0,-214.0,-0.187719,0.657626,0.940952,0.283327,0.430833,0.408691,0.585723,0.177032,0.433170,0.320175,0.240821,-0.079355,-0.247848,46000.0,91643.0,45643.0,0.992239,10.6,10.2,-0.4,-0.037736,False
1,1,0.441536,25613.0,Nell and Michael made my stay pleasant and eas...,Room in Historic East Nashville Craftsman Home,95811.0,36.17826,-86.74162,75.0,2.0,510.0,3.87,1.0,63.0,https://www.airbnb.com/rooms/25613,Serve up some sweet sounds on the piano and wi...,This home is in a walkable neighborhood with i...,2010-03-18,1.0,Private room in house,2.0,1 private bath,1.0,1.0,"[""Hair dryer"", ""Carbon monoxide alarm"", ""Air c...",2.0,1125.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,37206.0,4.703701e+10,412476.0,38.31,NaN,NaN,3.0,12.0,101.0,101.0,1048.0,2485.0,2525.0,40.0,0.016097,1895.0,1938.0,43.0,0.022691,1234.0,1092.0,-142.0,-0.115073,0.889738,0.930297,0.040559,0.045585,0.472296,0.742002,0.269707,0.571055,0.416532,0.274725,-0.141806,-0.340446,52372.0,116800.0,64428.0,1.230199,21.6,4.1,-17.5,-0.810185,True
2,6,0.495062,136015.0,"Very relaxing, beautiful property, and warm / ...",Apart. on the Ridge near Opryland,666322.0,36.19464,-86.67195,103.0,30.0,50.0,0.43,2.0,190.0,https://www.airbnb.com/rooms/136015,Fantastic View! Overlooks farms and is on the ...,Dead end street with great neighbors!,2011-06-05,4.0,Entire apartment,2.0,1 bath,1.0,1.0,"[""Fire extinguisher"", ""Wifi"", ""Dryer"", ""Washer...",30.0,1125.0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,37214.0,4.703702e+10,278031.0,65.43,1581.0,22.18,9.0,112.0,48.0,48.0,4429.0,5411.0,6024.0,613.0,0.113288,4451.0,4970.0,519.0,0.116603,2566.0,2714.0,148.0,0.057677,0.781186,0.760458,-0.020728,-0.026534,0.365311,0.430986,0.065675,0.179778,0.120811,0.179071,0.058261,0.482250,53590.0,68532.0,14942.0,0.278821,3.6,6.9,3.3,0.916667,False
3,1,0.460281,258817.0,Diana is such a gracious hostess. She has a b...,ButterflyRoom-private bath- open Jan 26,22296.0,36.16249,-86.58988,27.0,30.0,96.0,0.86,7.0,364.0,https://www.airbnb.com/rooms/258817,Morningstar House! Extended stay - 3+ months- ...,"I'm very near the Percy Priest Lake, natural ...",2009-06-19,16.0,Private

In [282]:
airbnb_topics_df = airbnb_topics[['listing_id', 'topic', 'GEOID']]

In [283]:
airbnb_topics_df.to_csv('../data/airbnb_topics_df.csv', index=False)

In [248]:
airbnb_topics_negative = airbnb_topics[airbnb_topics['topic']==1]

In [249]:
airbnb_topics_negative.value_counts('gentrifying')

gentrifying
False    17
True      4
dtype: int64

In [70]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

TypeError: '<' not supported between instances of 'int' and 'tuple'

## Find most representative Reviews for each Topic

In [ ]:
for i in range(6):
    for j in range(5):

        file = topics[topics.topic == i].sort_values('percent', ascending = False).head().iloc[j, 0]

        cmd = f'cp {file} representative_docs/topic_{i + 1}/.'

        os.system(cmd)

## Sample from Housing Project (this isn't working)

In [ ]:
# This is just a previous code, for safekeeping.
# Create a tokenizing function that takes text and removes all numbers
#def tokenizer(text):
#    return [x for x in re.findall(r'[a-z]+', text.lower()) if len(x) > 1]

In [ ]:
airbnb_train.head(1)

In [ ]:
# using a list comprehension, iterate over the four permit type contents and vectorize them using the TfidfVectorizer
vectorizer_test1 = TfidfVectorizer(
    tokenizer = tokenizer, 
    stop_words = 'english', 
    #min_df=50, 
    #max_df=0.4, 
    #ngram_range=(1,3)
).fit(airbnb_train)

In [ ]:
# Check to see the shape of the first sparse matrix generated
vectorizer_test1.transform(airbnb_train)

## Sample from Gensim Blog
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
test_comment = airbnb.comments_concatenated.iloc[2]

In [ ]:
print('original document: ')
words = []
for word in test_comment.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(test_comment))

## Try again from Michael's Notebook

In [ ]:
docs = airbnb.copy()['comments_concatenated'].iloc[0:3]

In [ ]:
docs[0]

In [ ]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in tqdm(range(len(docs))):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 2] for doc in docs]

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

## Rest of Michael's walkthrough below

In [ ]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
# Set training parameters.
num_topics = 6
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)